# Algorithm 12: Triangle Multiplication (Incoming Edges)

This is the companion to Algorithm 11. While the outgoing variant updates edge (i,j) using edges (i,k) and (j,k), the incoming variant uses edges (k,i) and (k,j).

## Algorithm Pseudocode

![Triangle Multiplication Incoming](../imgs/algorithms/TriangleMultiplicationIncoming.png)

## Source Code Location
- **File**: `AF2-source-code/model/modules.py`
- **Class**: `TriangleMultiplication` (same as Algorithm 11)
- **Lines**: 1250-1337
- **Key Difference**: `equation` config parameter: `'kjc,kic->ijc'` vs `'ikc,jkc->ijc'`

## Comparison: Outgoing vs Incoming

| Aspect | Outgoing (Alg 11) | Incoming (Alg 12) |
|--------|------------------|------------------|
| Equation | `'ikc,jkc->ijc'` | `'kjc,kic->ijc'` |
| Edge interpretation | (i→k), (j→k) | (k→i), (k→j) |
| Triangle vertex | k is the shared endpoint | k is the shared starting point |
| Information flow | From endpoints toward shared vertex | From shared vertex to endpoints |

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def layer_norm(x, axis=-1, eps=1e-5):
    mean = np.mean(x, axis=axis, keepdims=True)
    var = np.var(x, axis=axis, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)


def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))


def triangle_multiplication(pair_act, pair_mask, equation, num_intermediate=128):
    """
    Unified Triangle Multiplication for both outgoing and incoming.
    
    Args:
        pair_act: Pair activations [N_res, N_res, c_z]
        pair_mask: Pair mask [N_res, N_res]
        equation: Einsum equation string
            - 'ikc,jkc->ijc' for outgoing (Algorithm 11)
            - 'kjc,kic->ijc' for incoming (Algorithm 12)
        num_intermediate: Hidden dimension
    
    Returns:
        Updated pair activations [N_res, N_res, c_z]
    """
    N_res, _, c_z = pair_act.shape
    mask = pair_mask[:, :, None]
    
    # Layer norm
    act = layer_norm(pair_act, axis=-1)
    input_act = act
    
    # Left projection with gating
    left_proj_w = np.random.randn(c_z, num_intermediate) * 0.01
    left_gate_w = np.random.randn(c_z, num_intermediate) * 0.01
    left_gate_b = np.ones(num_intermediate)
    
    left_proj_act = mask * np.einsum('ijc,cd->ijd', act, left_proj_w)
    left_gate = sigmoid(np.einsum('ijc,cd->ijd', act, left_gate_w) + left_gate_b)
    left_proj_act = left_proj_act * left_gate
    
    # Right projection with gating
    right_proj_w = np.random.randn(c_z, num_intermediate) * 0.01
    right_gate_w = np.random.randn(c_z, num_intermediate) * 0.01
    right_gate_b = np.ones(num_intermediate)
    
    right_proj_act = mask * np.einsum('ijc,cd->ijd', act, right_proj_w)
    right_gate = sigmoid(np.einsum('ijc,cd->ijd', act, right_gate_w) + right_gate_b)
    right_proj_act = right_proj_act * right_gate
    
    # Triangle multiplication with specified equation
    act = np.einsum(equation, left_proj_act, right_proj_act)
    
    # Center layer norm + output projection
    act = layer_norm(act, axis=-1)
    output_w = np.random.randn(num_intermediate, c_z) * 0.01
    act = np.einsum('ijc,cd->ijd', act, output_w)
    
    # Output gating
    output_gate_w = np.random.randn(c_z, c_z) * 0.01
    output_gate_b = np.ones(c_z)
    gate = sigmoid(np.einsum('ijc,cd->ijd', input_act, output_gate_w) + output_gate_b)
    act = act * gate
    
    return act

## Test and Compare Both Variants

In [ ]:
# Test parameters
N_res = 32
c_z = 128

# Create test inputs
pair_act = np.random.randn(N_res, N_res, c_z).astype(np.float32)
pair_mask = np.ones((N_res, N_res), dtype=np.float32)

print(f"Input: {pair_act.shape}")
print()

In [ ]:
# Algorithm 11: Outgoing
print("Algorithm 11: Triangle Multiplication OUTGOING")
print("Equation: 'ikc,jkc->ijc'")
print("Meaning: For edge (i,j), aggregate over k using edges (i,k) and (j,k)")
print()

output_outgoing = triangle_multiplication(
    pair_act, pair_mask, 
    equation='ikc,jkc->ijc'
)

print(f"Output shape: {output_outgoing.shape}")
print(f"Stats: mean={output_outgoing.mean():.6f}, std={output_outgoing.std():.6f}")

In [ ]:
# Algorithm 12: Incoming
print("\nAlgorithm 12: Triangle Multiplication INCOMING")
print("Equation: 'kjc,kic->ijc'")
print("Meaning: For edge (i,j), aggregate over k using edges (k,j) and (k,i)")
print()

output_incoming = triangle_multiplication(
    pair_act, pair_mask,
    equation='kjc,kic->ijc'
)

print(f"Output shape: {output_incoming.shape}")
print(f"Stats: mean={output_incoming.mean():.6f}, std={output_incoming.std():.6f}")

In [ ]:
# Compare the two outputs
print("\nComparison:")
diff = np.abs(output_outgoing - output_incoming).mean()
print(f"Mean absolute difference: {diff:.6f}")
print("(Different equations produce different outputs as expected)")

## Geometric Interpretation

### Outgoing (Algorithm 11)
```
For edge (i → j):
   i ----→ k
   j ----→ k
   
Both i and j have edges pointing TO k.
Aggregate: sum over k of (edge i→k) * (edge j→k)
```

### Incoming (Algorithm 12)
```
For edge (i → j):
   k ----→ i
   k ----→ j
   
Both i and j receive edges FROM k.
Aggregate: sum over k of (edge k→j) * (edge k→i)
```

## Source Code Reference

```python
# From AF2-source-code/model/modules.py

class TriangleMultiplication(hk.Module):
  def __call__(self, act, mask, is_training=True):
    c = self.config
    
    # ... projections and gating ...
    
    # Key difference between algorithms:
    # "Outgoing" edges equation: 'ikc,jkc->ijc'
    # "Incoming" edges equation: 'kjc,kic->ijc'
    # Note on the Suppl. Alg. 11 & 12 notation:
    # For the "outgoing" edges, a = left_proj_act and b = right_proj_act
    # For the "incoming" edges, it's swapped:
    #   b = left_proj_act and a = right_proj_act
    act = jnp.einsum(c.equation, left_proj_act, right_proj_act)
    
    # ... output projection and gating ...
```

## Key Insights

1. **Complementary Views**: Outgoing and incoming capture different triangle relationships.

2. **Same Implementation**: Both use the same `TriangleMultiplication` class, just with different `equation` config.

3. **Sequential Application**: In Evoformer, outgoing is applied first, then incoming.

4. **O(N³) Complexity**: Both have O(N³ × c) complexity for the einsum operation.